# Segmenting and Clustering Neighborhoods in city of Toranto,Canada

1.Download and Explore Dataset

2.Explore Neighborhoods in Toranto, Canada

3.Analyze Each Neighborhood

4.Cluster Neighborhoods

5.Examine Clusters

## Install or Import required libraries to explore data

In [1]:

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')



Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1


In [2]:
!pip install bs4
from bs4 import BeautifulSoup
import requests

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=da60be8550a8c950998452b4126e1e3c52b72d431bee0a8a6d3feea2c99d47dd
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [3]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [4]:
html_data=requests.get(url).text

In [6]:
soup= BeautifulSoup(html_data, 'html5lib')

In [7]:
canada_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        canada_contents.append(cell)

In [8]:
canada_df=pd.DataFrame(canada_contents)

In [9]:
canada_contents[0]

{'Postal Code': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}

In [10]:
for data in canada_contents:
    postalcode=data["Postal Code"]
    borough  =data['Borough']
    neighborhood = data['Neighborhood']

    canada_df= canada_df.append({'Postal Code':postalcode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood  },
                                           ignore_index=True)

In [11]:
canada_df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [12]:
canada_df.shape

(206, 3)

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(canada_df['Borough'].unique()),
        canada_df.shape[0]
    )
)

The dataframe has 15 boroughs and 206 neighborhoods.


#### Here we are converting all the postal code of the canada_df  including with latitude and longitude

In [14]:
!pip install geocoder 

     |████████████████████████████████| 98 kB 9.6 MB/s  eta 0:00:01


In [15]:
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.arcgis('{}, Toronto, Ontario'.format('Postal Code'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print('The geograpical coordinate of  Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of  Toronto, Ontario are 43.648690000000045, -79.38543999999996.


#### Based on the given url  i am downloading the file  and then appending with the canada_df

In [16]:
url ="https://cocl.us/Geospatial_data"
geotable = pd.read_csv(url)
geotable.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
#append lat & long into table[0]
toronto=canada_df.merge(geotable , left_on='Postal Code', right_on='Postal Code')
toronto.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M3A,North York,Parkwoods,43.753259,-79.329656
2,M4A,North York,Victoria Village,43.725882,-79.315572
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
8,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
9,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
